In [1]:
import datetime
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard
from tqdm.keras import TqdmCallback
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler


/Users/akesgazepides/Recommendation_System_RecSys_Challenge_2024/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:

# Step 1: Create User DataFrame (user_df)
num_users = 1590
user_df = pd.DataFrame(np.random.uniform(low=-1, high=1, size=(num_users, 300)), 
                       index=np.arange(num_users), columns=np.arange(300))

In [3]:
user_df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.987036,0.548181,0.861899,0.949636,-0.264353,0.123444,-0.728850,-0.001582,0.869088,0.316835,...,-0.911315,-0.130398,0.824306,0.328267,0.763953,-0.299094,-0.663173,-0.785614,0.091697,0.022694
1,-0.035981,0.383406,0.355275,0.812436,-0.135662,0.869094,0.760887,0.573681,-0.800416,0.869356,...,0.163064,0.348463,0.723769,0.834318,0.130463,-0.944063,-0.024919,0.335105,-0.016085,0.966159


In [3]:

# Step 2: Create Article DataFrame (article_df)
num_articles = 4247
article_df = pd.DataFrame(np.random.uniform(low=-1, high=1, size=(num_articles, 300)), 
                          index=np.arange(num_articles), columns=np.arange(300))


In [5]:
article_df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.493883,-0.397649,0.246487,0.610953,-0.744756,0.378394,0.805706,-0.649033,-0.707469,0.658989,...,0.691331,0.977246,0.506997,0.250476,0.859761,-0.700932,-0.437441,0.140165,-0.557691,-0.076937
1,-0.021556,0.808796,0.930118,-0.106310,-0.362500,-0.848620,-0.860997,0.756616,-0.097077,-0.554402,...,0.093996,-0.748099,0.098246,0.929737,0.799071,-0.083675,0.074691,0.790937,0.268994,0.658648


In [4]:
# Step 3: Create Interaction Matrix DataFrame (interaction_matrix_df) minutes 
interaction_matrix_df = pd.DataFrame(np.random.randint(0, 101, size=(num_users, num_articles)), 
                                     index=np.arange(num_users), columns=np.arange(num_articles))

In [7]:
interaction_matrix_df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,4237,4238,4239,4240,4241,4242,4243,4244,4245,4246
0,12,33,100,95,95,86,91,15,19,74,...,65,21,36,68,75,66,36,6,46,61
1,64,13,64,25,76,8,9,51,28,79,...,89,60,7,37,77,87,98,87,100,100


In [5]:

# Step 4: Convert DataFrames to numpy arrays
user_vectors = user_df.values  # Shape: (1590, 300)
article_vectors = article_df.values  # Shape: (4247, 300)
interaction_matrix = interaction_matrix_df.values  # Shape: (1590, 4247)

In [6]:
# Normalize the original interaction matrix
interaction_matrix_normalized = MinMaxScaler().fit_transform(interaction_matrix)


In [8]:

# Step 5: Get indices of non-zero entries in the interaction matrix
user_idx, article_idx = np.where(interaction_matrix_normalized != 0)
read_times = interaction_matrix_normalized[user_idx[:20000], article_idx[:20000]] # choose 20000 because the result is 60000

In [9]:
# Check shapes before concatenation
print("user_vectors[user_idx] shape:", user_vectors[user_idx[:20000]].shape)
print("article_vectors[article_idx] shape:", article_vectors[article_idx[:20000]].shape)


user_vectors[user_idx] shape: (20000, 300)
article_vectors[article_idx] shape: (20000, 300)


In [10]:
# Step 6: Split the data into train and test sets with test size of 20%
X = np.hstack((user_vectors[user_idx[:20000]], article_vectors[article_idx[:20000]]))
y = read_times

In [11]:
print(X.shape)
print(y.shape)

(20000, 600)
(20000,)


In [12]:
sample = 100
X = X[:sample]
y = y[:sample]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (80, 600)
X_test shape: (20, 600)
y_train shape: (80,)
y_test shape: (20,)


In [15]:
# y_train = y_train.reshape(-1, 1)
# y_test = y_test.reshape(-1, 1)


In [16]:
# print("y_train shape:", y_train.shape)
# print("y_test shape:", y_test.shape)

In [14]:
# Define the model
model = Sequential([
    tf.keras.Input(shape=(600,)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

# Prepare TensorBoard callback
log_dir = "files/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
num_epochs = 2
history = model.fit(X_train, y_train, epochs=num_epochs, batch_size=16,
                    validation_data=(X_test, y_test),
                    callbacks=[tensorboard_callback, TqdmCallback(verbose=1)],
                    verbose=2)

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Epoch 1/2
